## Packages

In [ ]:
import random
import numpy as np
import os
import torch
import torch.nn as nn
#from pytorch_transformers import BertModel, BertTokenizer, BertConfig, WarmupLinearSchedule 
import re
import pandas as pd 
import json
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, SubsetRandomSampler
import pickle
from sklearn import metrics
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import average_precision_score
from scipy import stats

In [ ]:
from tqdm import tqdm_notebook, trange

def seed_everything(seed = 42): 
  random.seed(seed) 
  os.environ['PYTHONHASHSEED'] = str(seed) 
  np.random.seed(seed)
  torch.manual_seed(seed) 
  torch.cuda.manual_seed(seed) 
  torch.backends.cudnn.deterministic = True
# For reproducible results
seed_everything()

In [ ]:
import matplotlib as mpl
mpl.style.use('seaborn')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
%cd /content/gdrive/My Drive/seq

## Data Preprocessing

In [ ]:
class MyDataset(Dataset):
    def __init__(self, X, Y):
        self.data = X
        self.target = Y
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        return x, y
    
    def __len__(self):
        return len(self.data)

In [ ]:
%ls data/log

In [ ]:
npzfile = np.load('data/log/Astrocytes.npz')

In [ ]:
X, y = npzfile['arr_0'], npzfile['arr_1']

In [ ]:
subX, subY = shuffle(X, y, random_state=0)

In [ ]:
len(subY)

In [ ]:
testX = subX[int(len(subY)*0.8):]
testY = subY[int(len(subY)*0.8):]
validX = subX[int(len(subY)*0.6):int(len(subY)*0.8)]
validY = subY[int(len(subY)*0.6):int(len(subY)*0.8)]
trainX = subX[:int(len(subY)*0.6)]
trainY = subY[:int(len(subY)*0.6)]

### Convert to Torch Data

In [ ]:
train_X = torch.from_numpy(trainX)
train_y = torch.from_numpy(trainY)
valid_X  = torch.from_numpy(validX)
valid_y = torch.from_numpy(validY)
test_X = torch.from_numpy(testX)
test_y = torch.from_numpy(testY)

In [ ]:
train_dataset = MyDataset(train_X, train_y)
valid_dataset = MyDataset(valid_X, valid_y)
test_dataset = MyDataset(test_X, test_y)

## Helper Functions

In [ ]:
def bestmodel(model_name,save_model_time,valid_loss):
    bestloss = 10000
    if valid_loss < bestloss :
        bestloss = valid_loss
        torch.save(model_name, 'model/model{save_model_time}/bestmodel.pkl'.format(save_model_time=save_model_time))
        torch.save(model_name.state_dict(), 'model/model{save_model_time}/net_params_bestmodel.pkl'.format(save_model_time=save_model_time))
    return True  

## Training and Validating

In [ ]:
save_model_time = '0'
mkpath = 'model/model%s'% save_model_time
# os.makedirs(mkpath)

In [ ]:
class TrainHelper():
    '''
    Helper class that makes it a bit easier and cleaner to define the training routine
    
    '''

    def __init__(self,model,train_set,test_set,opts):
      self.model = model  # neural net

      # device agnostic code snippet
      self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
      self.model.to(self.device)

      self.epochs = opts['epochs']
      self.optimizer = torch.optim.Adam(model.parameters(), opts['lr']) # optimizer method for gradient descent
      #self.optimizer = torch.optim.SGD(model.parameters(), opts['lr'])
      self.criterion = torch.nn.MSELoss()
      self.train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                      batch_size=opts['batch_size'],
                                                      shuffle=True)
      self.valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                                      batch_size=opts['batch_size'],
                                                      shuffle=True)
    def train(self):
      self.model.train() # put model in training mode
      for epoch in range(self.epochs):
          self.tr_loss = []
          for i, (data,labels) in tqdm_notebook(enumerate(self.train_loader),
                                                  total = len(self.train_loader)):

              data, labels = data.to(self.device),labels.to(self.device)
              self.optimizer.zero_grad()  
              outputs = self.model(data)
              labels = labels.unsqueeze(1)
              loss = self.criterion(outputs.float(), labels.float())
              loss.backward()                        
              self.optimizer.step()                  
              self.tr_loss.append(loss.item())       
          if (epoch+1) % 5 == 0 or epoch == 0: # save the model every _ epoch
              torch.save(self.model, 'model/model{save_model_time}/net_{epoch}.pkl'.format(save_model_time=save_model_time,epoch=int((epoch+1)/5)))
              torch.save(self.model.state_dict(), 'model/model{save_model_time}/net_params_{epoch}.pkl'.format(save_model_time=save_model_time,epoch=int((epoch+1)/5)))
          
          self.test(epoch) # run through the validation set

    def test(self,epoch):
            
      self.model.eval()    # puts model in eval mode
      self.test_loss = []
      self.test_accuracy = []

      for i, (data, labels) in enumerate(self.valid_loader):
          
          data, labels = data.to(self.device),labels.to(self.device)
          # pass data through network
          # turn off gradient calculation to speed up calcs and reduce memory
          with torch.no_grad():
              outputs = self.model(data)
          # make our predictions and update our loss info
          labels = labels.unsqueeze(1)
          loss = self.criterion(outputs, labels)
          self.test_loss.append(loss.item())
      
      test_loss.append(np.mean(self.test_loss))
      train_loss.append(np.mean(self.tr_loss))    
      bestmodel(self.model,save_model_time,np.mean(self.test_loss)) # find best model
      print('epoch: {}, train loss: {}, test loss: {}'.format( 
      epoch+1, np.mean(self.tr_loss), np.mean(self.test_loss)))

## Testing

In [ ]:
train_X, train_y = shuffle(train_X, train_y, random_state=0) 
train_X_sub = train_X[:2000]
train_y_sub = train_y[:2000]
sub_dataset = MyDataset(train_X_sub, train_y_sub)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=True)
sub_loader = torch.utils.data.DataLoader(sub_dataset, batch_size=100, shuffle=True)

In [ ]:
def get_list_con(model, loader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    pred, true = [], []
    for i, (data, labels) in enumerate(loader):
      data, labels = data.to(device),labels.to(device)
    # pass data through network
    # turn off gradient calculation to speed up calcs and reduce memory
      with torch.no_grad():
          outputs = model(data)
    # make our predictions and update our loss info
      predicted = []
      for o in outputs.tolist():
        predicted.append(o[0])
      pred.extend(predicted)
      true.extend(labels.tolist())
    return true, pred

### AUC

In [ ]:
def getAUC(model):
    labels_tr, predicts_tr = get_list_cat(model, sub_loader)
    score_tr = metrics.roc_auc_score(labels_tr, predicts_tr, average='weighted')
    labels_ts, predicts_ts = get_list_cat(model, test_loader)
    score_ts = metrics.roc_auc_score(labels_ts, predicts_ts, average='weighted')
    return score_tr, score_ts

### AUPRC

In [ ]:
def getAUPRC(model):
    labels, predicts = get_list_con(model)
    auprc = average_precision_score(labels, predicts)
    return auprc

### Pearson R

In [ ]:
def getR(model):
    labels_tr, predicts_tr = get_list_con(model, sub_loader)
    corr_tr, _ = stats.pearsonr(labels_tr, predicts_tr)
    labels_ts, predicts_ts = get_list_con(model, test_loader)
    corr_ts, _ = stats.pearsonr(labels_ts, predicts_ts)
    return corr_tr, corr_ts

### Average Percentage Change

In [ ]:
def avgDiff(model):
    labels, predicts = get_list_con(model, test_loader)
    all = []
    for i, y in enumerate(labels):
      div = y
      if y == 0:
        div = 0.0000000001
      all.append((predicts[i]-y)/div)
    all = np.array(all)
    all_abs = np.absolute(all)
    return np.mean(all_abs)

### Plot Train Verse Test Loss

In [ ]:
def pltloss(train_loss, test_loss, epoch):
    epochs = [i for i in range(epoch)]
    fig = plt.figure()
    plt.plot(epochs, train_loss, 'g', label='Training loss')
    plt.plot(epochs, test_loss, 'b', label='Testing loss')
    plt.title('Training and Testing Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

### Plot R

In [ ]:
def pltR(r_tr, r_ts, epoch):
    epochs = [i for i in range(epoch+1)][::5][1:]
    fig = plt.figure()
    plt.plot(epochs, r_tr, 'g', label='Pearson R for Training')
    plt.plot(epochs, r_ts, 'b', label='Pearson R for Testing')
    plt.title('R Score Over Time')
    plt.xlabel('Epochs')
    plt.ylabel('R')
    plt.legend()
    plt.show()

### Plot Predicated Verse Label

In [ ]:
def plotcomp(model, loader):
    labels, predicts = get_list_con(model, loader)
    # idx_list = [i for i in range(len(labels))]
    # idx_sele = random.sample(idx_list, 50)
    fig = plt.figure()
    # label_sele, pred_sele = [], []
    # for i in idx_sele:
    #   label_sele.append(labels[i])
    #   pred_sele.append(predicts[i])
    # plt.scatter(pred_sele, label_sele, c='b', marker='+')
    plt.scatter(labels, predicts)
    #l = max(max(pred_sele), max(label_sele))
    l = max(max(predicts), max(labels))
    s = min(min(predicts), min(labels))
    plt.plot([s, l], [s, l], color = 'black', linewidth = 1)
    plt.title('Actual Values vs Predicated Values')
    plt.xlabel('Predicated Values')
    plt.ylabel('Actual Values')
    plt.xlim(s, l)
    plt.ylim(s, l)
    plt.show()

## Models

### CNN

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_size):
        """
        init convolution and activation layers
        Args:
        x: (Nx1x2004)
        class: 

        """
        super(CNN, self).__init__() 
        
        self.conv1 = torch.nn.Conv1d(input_size[0], 128, 2)
        self.relu = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv1d(128, 64, 2)
        self.pool = torch.nn.AvgPool1d(4)
        self.fc1 = torch.nn.Linear(2368, 2368)
        self.fc2 = torch.nn.Linear(2368, 1)

    def forward(self, x):
        """
        forward function describes how input tensor is transformed to output tensor
        Args:
            
        """
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

In [ ]:
cnn = CNN(train_X.shape[1:])
cnn

In [ ]:
opts = {
    'lr': 1e-4,
    'epochs': 50,
    'batch_size': 100,
}

In [ ]:
test_loss, train_loss = [], []
CNNTrainer = TrainHelper(model = cnn,
                      train_set = train_dataset,
                      test_set = valid_dataset, opts = opts)

In [ ]:
CNNTrainer.train()

#### Check for Output

In [ ]:
r_list = []
for num in range(opts['epochs']//5):
  cnn.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  cnn.cuda()
  r_list.append(getR(cnn))

In [ ]:
pltloss(train_loss, test_loss, opts['epochs'])

In [ ]:
pltR(r_list, opts['epochs'])

In [ ]:
plotcomp(cnn)

### Basenji

https://github.com/calico/basenji/blob/master/manuscripts/genome_research2018/params.txt

In [ ]:
class Besenji(nn.Module):
    def __init__(self, input_size):
        """
        init convolution and activation layers
        Args:
        class: 

        """
        super(Besenji, self).__init__() 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.05)
        self.dropout1 = nn.Dropout(p=0.1)
        self.batch6 = nn.BatchNorm1d(108)

        self.conv1 = nn.Conv1d(input_size[0], 312, kernel_size=22)
        self.batch1 = nn.BatchNorm1d(312)

        self.conv2 = nn.Conv1d(312, 368, kernel_size=1)
        self.pool2 = nn.MaxPool1d(2)
        self.batch2 = nn.BatchNorm1d(368)

        self.conv3 = nn.Conv1d(368, 435, kernel_size=6)
        self.pool3 = nn.MaxPool1d(4)
        self.batch3 = nn.BatchNorm1d(435)

        self.conv4 = nn.Conv1d(435, 607, kernel_size=6)
        self.pool4 = nn.MaxPool1d(4)
        self.batch4 = nn.BatchNorm1d(607)
        
        self.conv5 = nn.Conv1d(607, 717, kernel_size=3)
        self.batch5 = nn.BatchNorm1d(717)

        self.conv6 = nn.Conv1d(717, 108, kernel_size=3, dilation=2)

        self.conv7 = nn.Conv1d(108, 108, kernel_size=3, dilation=4)

        self.conv8 = nn.Conv1d(108, 108, kernel_size=3, dilation=8)

        self.conv9 = nn.Conv1d(108, 108, kernel_size=3, dilation=16)
        
        self.conv10 = nn.Conv1d(108, 108, kernel_size=3, dilation=32)

        self.conv11 = nn.Conv1d(108, 108, kernel_size=3, dilation=64)

        self.conv12 = nn.Conv1d(108, 1365, kernel_size=1)
        self.batch12 = nn.BatchNorm1d(1365)

        self.conv13 = nn.Conv1d(1365, 1, kernel_size=1)

    def forward(self, x):
        x = self.batch1(self.conv1(x))
        x = self.relu(x)
        x = self.dropout(x)

        x = self.relu(self.batch2(self.conv2(x)))
        x = self.pool2(x)
        x = self.dropout(x)

        x = self.relu(self.batch3(self.conv3(x)))
        x = self.pool3(x)
        x = self.dropout(x)

        x = self.relu(self.batch4(self.conv4(x)))
        x = self.pool4(x)
        x = self.dropout(x)
        
        x = self.relu(self.batch5(self.conv5(x)))
        x = self.dropout(x)

        x = self.relu(self.batch6(self.conv6(x)))
        x = self.dropout1(x)

        x = self.relu(self.batch6(self.conv7(x)))
        x = self.dropout1(x)

        x = self.relu(self.batch6(self.conv8(x)))
        x = self.dropout1(x)

        x = self.relu(self.batch6(self.conv9(x)))
        x = self.dropout1(x)
        
        x = self.relu(self.batch6(self.conv10(x)))
        x = self.dropout1(x)

        x = self.relu(self.batch6(self.conv11(x)))
        x = self.dropout1(x)

        x = self.relu(self.batch12(self.conv12(x)))
        x = self.dropout(x)
        
        x = self.conv13(x)

        return x

In [ ]:
basenji = Besenji(train_X.shape[1:])
basenji

In [ ]:
opts = {
    'lr': 1e-4,
    'epochs': 50,
    'batch_size': 100,
}

In [ ]:
test_loss, train_loss = [], []
BasenjiTrainer = TrainHelper(model = basenji,
                      train_set = train_dataset,
                      test_set = valid_dataset, opts = opts)

In [ ]:
BasenjiTrainer.train()

### VGG

In [ ]:
class VGG16(nn.Module):
    def __init__(self, input_size):
        super(VGG16, self).__init__()
        self.conv1_1 = nn.Conv1d(input_size[0], 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv1d(64, 64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv1d(128, 128, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv1d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv1d(256, 256, kernel_size=3, padding=1)

        self.conv4_1 = nn.Conv1d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv1d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv1d(512, 512, kernel_size=3, padding=1)

        self.conv5_1 = nn.Conv1d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv1d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv1d(512, 512, kernel_size=3, padding=1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        # max pooling (kernel_size, stride)
        self.pool = nn.MaxPool1d(2, 2)

        # fully conected layers
        self.fc6 = nn.Linear(512*18, 1000)
        # self.fc6 = nn.Linear(512*7, 1000)
        self.fc7 = nn.Linear(1000, 100)
        self.fc8 = nn.Linear(100, 1)

    def forward(self, x):
        x = self.conv1_1(x)
        x = self.relu(x)
        x = self.conv1_2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv2_1(x)
        x = self.relu(x)
        x = self.conv2_2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv3_1(x)
        x = self.relu(x)
        x = self.conv3_2(x)
        x = self.relu(x)
        x = self.conv3_3(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv4_1(x)
        x = self.relu(x)
        x = self.conv4_2(x)
        x = self.relu(x)
        x = self.conv4_3(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.relu(self.conv5_1(x))
        x = self.relu(self.conv5_2(x))
        x = self.relu(self.conv5_3(x))
        x = self.pool(x)
        # print(x.size())
        x = torch.flatten(x, 1)
        # print(x.size())
        # assert 0
        x = self.fc6(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc7(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc8(x)

        return x

In [ ]:
vgg = VGG16(train_X.shape[1:])
vgg

In [ ]:
opts = {
    'lr': 1e-4,
    'epochs': 50,
    'batch_size': 100,
}

In [ ]:
test_loss, train_loss = [], []
VGGTrainer = TrainHelper(model = vgg,
                      train_set = train_dataset,
                      test_set = valid_dataset, opts = opts)

In [ ]:
VGGTrainer.train()

#### Check for Output

In [ ]:
r_list_ts = []
r_list_tr = []
for num in range(opts['epochs']//5):
  vgg.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  vgg.cuda()
  tr, ts = getR(vgg)
  r_list_ts.append(ts)
  r_list_tr.append(tr)

In [ ]:
vgg.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_8.pkl'))
getR(vgg)

In [ ]:
avgDiff(vgg)

In [ ]:
pltloss(train_loss, test_loss, opts['epochs'])

In [ ]:
pltR(r_list_tr, r_list_ts, opts['epochs'])

In [ ]:
plotcomp(vgg,sub_loader)